<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>

# 4. 无分类器扩散引导

到目前为止，我们已经能够使用 [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) 数据集训练模型来生成服装图像。但是，用户无法指定应生成哪种图像。让我们通过使用 [无分类器扩散引导](https://arxiv.org/pdf/2207.12598.pdf) 来解决这个问题，这是一种创建 [条件扩散模型](https://github.com/TeaPearce/Conditional_Diffusion_MNIST/tree/main) 的相对简单的方法。

#### 学习目标

本笔记本的目标是：
* 向 U-Net 添加分类嵌入
* 使用伯努利掩码训练模型
* 添加反向扩散过程的权重
* 在更具挑战性的数据集上练习

在开始之前，让我们加载必要的库和数据集信息。

In [ ]:
import glob
import torch
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.transforms as transforms

# Visualization tools
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.utils import save_image, make_grid

# User defined libraries
from utils import other_utils
from utils import ddpm_utils
from utils import UNet_utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
IMG_SIZE = 16
IMG_CH = 1
BATCH_SIZE = 128
N_CLASSES = 10
data, dataloader = other_utils.load_transformed_fashionMNIST(IMG_SIZE, BATCH_SIZE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 4.1 模型训练

我们之前的 Beta 计划将保持不变。

In [ ]:
nrows = 10
ncols = 15

T = nrows * ncols
B_start = 0.0001
B_end = 0.02
B = torch.linspace(B_start, B_end, T).to(device)
ddpm = ddpm_utils.DDPM(B, device)

这次，我们的 `U-Net` 略有不同。我们添加了一些更改，并将 U-Net 架构移至其自己的 [UNet_utils.py](utils/UNet_utils.py) 文件中。

在 `__init__` 函数中，我们添加了一个新参数：`c_embed_dim`。与时间步长 `t` 一样，我们可以为我们的类类别创建嵌入。

```python
        self.sinusoidaltime = SinusoidalPositionEmbedBlock(t_embed_dim)
        self.t_emb1 = EmbedBlock(t_embed_dim, up_chs[0])
        self.t_emb2 = EmbedBlock(t_embed_dim, up_chs[1])
        self.c_embed1 = EmbedBlock(c_embed_dim, up_chs[0])  # New
        self.c_embed2 = EmbedBlock(c_embed_dim, up_chs[1])  # New
```

接下来，在 `forward` 函数中，我们有两个新参数：`c` 和 `c_mask`。
* `c` 是一个表示我们的分类输入的向量。它可以是 [one-hot 编码](https://www.kaggle.com/code/dansbecker/using-categorical-data-with-one-hot-encoding) 或嵌入向量。
* `c_mask` 用于将 `c` 内的值随机设置为零。这有助于模型了解在没有分类输入的情况下平均输出可能是什么，就像在之前的笔记本中一样。

```python
        c = c * c_mask
        c_emb1 = self.c_embed1(c)
        c_emb2 = self.c_embed2(c)
```

我们可以通过多种不同的方式将这些嵌入的分类信息组合到模型中。一种流行的方法是使用 [缩放和移位](https://arxiv.org/pdf/2210.08823.pdf)。我们可以将我们的分类嵌入缩放（乘以）到潜在图像，然后（添加）我们的时间嵌入 `t_emb`。尺度和移位分别充当方差和平均值的一种形式。

```python
        up0 = self.up0(latent_vec)
        up1 = self.up1(c_emb1 * up0 + t_emb1, down2)  # Changed
        up2 = self.up2(c_emb2 * up1 + t_emb2, down1)  # Changed
        return self.out(torch.cat((up2, down0), 1))
```

我们将使用下面的 `get_context_mask` 函数将标签转换为模型可以处理的格式。由于我们的标签是一个整数，我们可以使用 [F.one_hot](https://pytorch.org/docs/stable/generated/torch.nn. functional.one_hot.html) 将其转换为编码向量。

要将此独热编码的值随机设置为零，我们可以使用 [伯努利](https://mathworld.wolfram.com/BernoulliDistribution.html) 分布。此分布就像抛掷加权硬币。“正面”的概率为 $p$%，而“反面”的概率为 $1-p$%。在本例中，我们的 `drop_prob` 代表“反面”。

<center><img src="images/bernoulli.png"/></center>

In [ ]:
def get_context_mask(c, drop_prob):
    c_hot = F.one_hot(c.to(torch.int64), num_classes=N_CLASSES).to(device)
    c_mask = torch.bernoulli(torch.ones_like(c_hot).float() - drop_prob).to(device)
    return c_hot, c_mask

这就是我们需要添加到 `UNet` 中以便它从分类数据中学习的所有更改。让我们继续构建这个新结构的实例。

In [ ]:
model = UNet_utils.UNet(
    T, IMG_CH, IMG_SIZE, down_chs=(64, 64, 128), t_embed_dim=8, c_embed_dim=N_CLASSES
)
print("Num params: ", sum(p.numel() for p in model.parameters()))
model = torch.compile(model.to(device))

为了说明模型试图生成什么，让我们跟踪不同的类名。这里的顺序与数据集的标签顺序相匹配。例如，当标签为 3 时，它代表一件连衣裙。

In [ ]:
class_names = [
    "Top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

我们的训练步骤与上次几乎相同。我们将在每个预览步骤中循环遍历每个类别，以便我们可以看到模型在所有类别中的学习情况。

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 3
preview_c = 0

model.train()
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        c_drop_prob = 0.1
        optimizer.zero_grad()

        t = torch.randint(0, T, (BATCH_SIZE,), device=device).float()
        x = batch[0].to(device)
        c_hot, c_mask = get_context_mask(batch[1], c_drop_prob)  # New
        loss = ddpm.get_loss(model, x, t, c_hot, c_mask)
        loss.backward()
        optimizer.step()

        if epoch % 1 == 0 and step % 100 == 0:
            class_name = class_names[preview_c]
            print(f"Epoch {epoch} | Step {step:03d} | Loss: {loss.item()} | C: {class_name}")
            c_drop_prob = 0 # Do not drop context for preview
            c_hot, c_mask = get_context_mask(torch.Tensor([preview_c]), c_drop_prob)
            ddpm.sample_images(model, IMG_CH, IMG_SIZE, ncols, c_hot, c_mask)
            preview_c = (preview_c + 1) % N_CLASSES

效果怎么样？尝试运行下面的代码单元来查看每个类的最终结果。

In [ ]:
plt.figure(figsize=(8,8))
ncols = 3
c_drop_prob = 0 # Change me to a value between 1 and 0

for c in range(10):
    print(class_names[c])
    c_hot, c_mask = get_context_mask(torch.Tensor([c]), c_drop_prob)
    ddpm.sample_images(model, IMG_CH, IMG_SIZE, ncols, c_hot, c_mask, axis_on=True)

## 4.2 调节逆向扩散

还不错，但也不是很好。有些类别仍然相互影响。例如，这只鞋似乎有一只衬衫袖子挂在上面。

<center><img src="images/shirt_shoe.png" alt="shirt_shoe" style="width: 400px;"/></center>

我们可以通过增加类别的“权重”来解决这个问题。以下是策略：
* 在每个时间步的反向扩散过程中，我们将对图像进行**两次**去噪
  * 我们提取的第一张噪声图像将**保留**其分类器信息（`e_t_keep_c`）
  * 我们提取的第二张噪声图像将**删除**其分类器信息（`e_t_drop_c`）
* 我们将使用以下方法从分类噪声中减去平均噪声：
  * `e_t = (1 + w) * e_t_keep_c - w * e_t_drop_c`
  * 其中 `w` 是我们选择作为超参数的权重值
* 我们将使用这个新的 `e_t` 噪声通过 `reverse_q` 执行扩散。
* 重复上述步骤，从 `t` = `T` 到 `0`

<center><img src="images/weighted_reverse_diffusion.png"/></center>

我们在下面的 `sample_w` 中定义了这些步骤。 `sample_w` 采用噪声权重列表，以便我们可以比较它如何影响扩散结果。

In [ ]:
@torch.no_grad()
def sample_w(
    model, input_size, T, c, w_tests=[-2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0], store_freq=10
):
    # Preprase "grid of samples" with w for rows and c for columns
    n_samples = len(w_tests) * len(c)

    # One w for each c
    w = torch.tensor(w_tests).float().repeat_interleave(len(c))
    w = w[:, None, None, None].to(device)  # Make w broadcastable
    x_t = torch.randn(n_samples, *input_size).to(device)

    # One c for each w
    c = c.repeat(len(w_tests), 1)

    # Double the batch
    c = c.repeat(2, 1)

    # Don't drop context at test time
    c_mask = torch.ones_like(c).to(device)
    c_mask[n_samples:] = 0.0

    x_t_store = []
    for i in range(0, T)[::-1]:
        # Duplicate t for each sample
        t = torch.tensor([i]).to(device)
        t = t.repeat(n_samples, 1, 1, 1)

        # Double the batch
        x_t = x_t.repeat(2, 1, 1, 1)
        t = t.repeat(2, 1, 1, 1)

        # Find weighted noise
        e_t = model(x_t, t, c, c_mask)
        e_t_keep_c = e_t[:n_samples]
        e_t_drop_c = e_t[n_samples:]
        e_t = (1 + w) * e_t_keep_c - w * e_t_drop_c

        # Deduplicate batch for reverse diffusion
        x_t = x_t[:n_samples]
        t = t[:n_samples]
        x_t = ddpm.reverse_q(x_t, t, e_t)

        # Store values for animation
        if i % store_freq == 0 or i == T or i < 10:
            x_t_store.append(x_t)

    x_t_store = torch.stack(x_t_store)
    return x_t, x_t_store

是时候看看它的实际效果了！运行下面的代码，生成一些衣服，并使用 [make_grid](https://pytorch.org/vision/main/generated/torchvision.utils.make_grid.html) 将它们排列成网格。

In [ ]:
c = torch.arange(N_CLASSES).to(device)
c_drop_prob = 0  # Keep all category information for sampling
c_hot, c_mask = get_context_mask(c, c_drop_prob)
input_size = (IMG_CH, IMG_SIZE, IMG_SIZE)

x_0, x_t_store = sample_w(model, input_size, T, c_hot)

In [ ]:
grids = [other_utils.to_image(make_grid(x_t.cpu(), nrow=N_CLASSES)) for x_t in x_t_store]
other_utils.save_animation(grids, "04_images/fashionMNIST.gif")

点击 [此处](04_images/fashionMNIST.gif) 可查看刚刚生成的反向扩散过程的动画。

行表示从 `[-2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0]` 开始增加的 `w` 值。前两行是负数，这意味着它更强调模型的平均值而不是分类值。该模型有时会生成与预期完全不同的服装。最后几行在生成与其标签匹配的物品方面是一致的。

## 4.3 改良 TF Flowers

恭喜您取得如此大的成就！您已经掌握了 FashionMNIST，现在是时候迎接更大的挑战了：彩色图像。对于这个挑战，我们将使用 [TF Flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers) 数据集的修改版本。

这些图像已略作修改，以便生成图像。例如，用户“_e.t”拍摄的这张照片已被裁剪以聚焦花朵。

<center>
<img src="images/24459750_eb49f6e4cb_m.jpg";/>
<img src="data/cropped_flowers/sunflowers/24459750_eb49f6e4cb_m.jpg";/>
</center>

由于颜色增加了一个维度，模型的训练时间会更长。为了加快速度，让我们将图像预加载到 GPU 上。如果我们在将它们加载到我们的 GPU 之前 [调整大小](https://pytorch.org/vision/stable/generated/torchvision.transforms.Resize.html)，它们将不会占用太多空间。

我们可以使用这种技术，因为数据集相对较小。对于较大的数据集，这可能不可行。

首先，我们应该定义数据集变量：
* 图像宽度和高度 `IMG_SIZE`
* 图像通道数 `IMG_CH`
* 批次大小 `BATCH_SIZE`
* 生成图像的大小 `INPUT_SIZE`

In [ ]:
IMG_SIZE = 32
IMG_CH = 3
BATCH_SIZE = 128
INPUT_SIZE = (IMG_CH, IMG_SIZE, IMG_SIZE)

为了将图像存储在 GPU 上，我们将有一个 `pre_transforms` 列表，它将在我们初始化数据集时运行一次。然后，`random_transforms` 将在从数据集中提取每个批次时在其上运行。

[Resize](https://pytorch.org/vision/stable/generated/torchvision.transforms.Resize.html) 将调整图像大小，以便宽度和高度之间的较小边缘与我们指定的大小相匹配。然后，我们可以使用 [RandomCrop](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomCrop.html) 使图像变成正方形，并通过随机数据增强有效地增加数据集的大小。

In [ ]:
import torchvision
from torch.utils.data import Dataset, DataLoader

pre_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),  # Scales data into [0,1]
    transforms.Lambda(lambda t: (t * 2) - 1)  # Scale between [-1, 1]
])

random_transforms = transforms.Compose([
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
])

接下来，让我们开发一个函数来读取图像文件。我们将直接使用每个图像的父级来识别其标签。我们在下面的 `DATA_LABELS` 中列出了三个类别。

<center>
    <img src="data/cropped_flowers/daisy/14219214466_3ca6104eae_m.jpg";/>
    <img src="data/cropped_flowers/sunflowers/1240625276_fb3bd0c7b1.jpg";/>
    <img src="data/cropped_flowers/roses/14510185271_b5d75dd98e_n.jpg";/>
</center>

从左到右，我们有 Allison Brown 的 `雏菊` 、Rob Young 的 `向日葵` 和 Matteo Accattino 的 `玫瑰` 。每张照片的作者都列在 [LICENSE.txt](data/cropped_flowers/LICENSE.txt) 中

让我们使用 [glob](https://docs.python.org/3/library/glob.html) 函数以编程方式获取每张花卉照片的数据路径。

In [ ]:
DATA_DIR = "data/cropped_flowers/"
DATA_LABELS = ["daisy", "sunflowers", "roses"]
N_CLASSES = len(DATA_LABELS)

data_paths = glob.glob(DATA_DIR + DATA_LABELS[0] + '/*.jpg', recursive=True)
data_paths[:5]  # First 5 paths

我们可以使用 pytorch 的 [Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) 工具来创建我们自己的数据集。`__init__` 将在初始化类时运行一次。`__getitem__` 在随机应用我们的 `random_transforms` 时返回我们的图像和标签。

In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        self.imgs = []
        self.labels = []
        
        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(DATA_DIR + label + '/*.jpg', recursive=True)
            for path in data_paths:
                img = Image.open(path)
                self.imgs.append(pre_transforms(img).to(device))
                self.labels.append(l_idx)


    def __getitem__(self, idx):
        img = random_transforms(self.imgs[idx])
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

train_data = MyDataset()
dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

是时候初始化我们的 U-Net 了。它与之前相同，但我们的 `T` 和 `down_chs` 要大得多。

In [ ]:
T = 400
B_start = 0.0001
B_end = 0.02
B = torch.linspace(B_start, B_end, T).to(device)
ddpm = ddpm_utils.DDPM(B, device)

In [ ]:
model_flowers = UNet_utils.UNet(
    T, IMG_CH, IMG_SIZE, down_chs=(256, 256, 512), t_embed_dim=8, c_embed_dim=N_CLASSES
)
print("Num params: ", sum(p.numel() for p in model_flowers.parameters()))
model_flowers = torch.compile(model_flowers.to(device))

让我们创建一个函数来采样我们的扩散模型，以便我们可以在训练期间和之后生成图像。

In [ ]:
def sample_flowers(n_classes):
    c_test = torch.arange(n_classes).to(device)
    c_hot_test, c_mask_test = get_context_mask(c_test, 0)
    x_gen, x_gen_store = sample_w(model_flowers, INPUT_SIZE, T, c_hot_test)
    return x_gen, x_gen_store

**TODO** 以下是我们新的训练循环。我们已将其转换为函数，但尚未完成。每个 `FIXME` 应替换为以下之一：
* `dataloader`
* `epochs`
* `n_classes`
* `c_drop_prob`

如果需要，请单击 `...` 获取解决方案。

In [ ]:
def train_flowers(dataloader, epochs=100, n_classes=N_CLASSES, c_drop_prob=0.1, save_dir = "04_images/"):
    lrate = 1e-4
    optimizer = torch.optim.Adam(model_flowers.parameters(), lr=lrate)

    model_flowers.train()
    for epoch in range(FIXME):
        for step, batch in enumerate(FIXME):
            optimizer.zero_grad()
            t = torch.randint(0, T, (BATCH_SIZE,), device=device).float()
            x = batch[0].to(device)
            c_hot, c_mask = get_context_mask(batch[1], FIXME)
            loss = ddpm.get_loss(model_flowers, x, t, c_hot, c_mask)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch} | Step {step:03d} | Loss: {loss.item()}")
        if epoch % 5 == 0 or epoch == int(epochs - 1):
            x_gen, x_gen_store = sample_flowers(FIXME)
            grid = make_grid(x_gen.cpu(), nrow=n_classes)
            save_image(grid, save_dir + f"image_ep{epoch:02}.png")
            print("saved images in " + save_dir + f" for episode {epoch}")

In [ ]:
def train_flowers(dataloader, epochs=100, n_classes=N_CLASSES, c_drop_prob=0.1, save_dir = "04_images/"):
    lrate = 1e-4
    optimizer = torch.optim.Adam(model_flowers.parameters(), lr=lrate)

    c = torch.arange(n_classes).to(device)
    c_hot_test, c_mask_test = get_context_mask(c, 0)

    model_flowers.train()
    for epoch in range(epochs):
        for step, batch in enumerate(dataloader):
            optimizer.zero_grad()
            t = torch.randint(0, T, (BATCH_SIZE,), device=device).float()
            x = batch[0].to(device)
            c_hot, c_mask = get_context_mask(batch[1], c_drop_prob)
            loss = ddpm.get_loss(model_flowers, x, t, c_hot, c_mask)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch} | Step {step:03d} | Loss: {loss.item()}")
        if epoch % 5 == 0 or epoch == int(epochs - 1):
            x_gen, x_gen_store = sample_flowers(n_classes)
            grid = make_grid(x_gen.cpu(), nrow=n_classes)
            save_image(grid, save_dir + f"image_ep{epoch:02}.png")
            print("saved images in " + save_dir + f" for episode {epoch}")

关键时刻！让我们看看扩散模型如何处理彩色图像。该模型大约需要 **十五分钟** 进行训练。一旦开始训练，就可以喝杯咖啡、茶或自己选择的舒适饮料。然后，查看 `04_images` 目录中的测试输出。第 50 个 epoch ，花朵已经可以识别，第 100 个 epoch ，模型将真正达到最佳状态。

In [ ]:
train_flowers(dataloader)

由于扩散模型的随机性，有些图像会比其他图像更好。尝试重新采样，直到得到您想要保留的输出作为纪念品。然后，运行单元以将其转换为动画。

In [ ]:
model.eval()
x_gen, x_gen_store = sample_flowers(N_CLASSES)
grid = make_grid(x_gen.cpu(), nrow=N_CLASSES)
other_utils.show_tensor_image([grid])
plt.show()

In [ ]:
grids = [other_utils.to_image(make_grid(x_gen.cpu(), nrow=N_CLASSES)) for x_gen in x_gen_store]
other_utils.save_animation(grids, "04_images/flowers.gif")

## 4.4 下一步

十几分钟的训练就取得了不错的成绩！在下一篇笔记本中，我们将使用完整的文本到图像管道使结果更加出色。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>